In [1]:
import datetime
import matplotlib
import numpy as np
import pandas as pd
import pyreadr
import pickle

import os

In [25]:
datasets = [1,2,3,4,5]
d=1

In [31]:
for d in datasets:
    file = '../../data/app-level/old/app_all_' + str(d) + '.csv'
    df = pd.read_csv(file, sep = ';')
    df['model'] = df['Metrics'].apply(lambda x: x.split('-')[0])
    print(df.sort_values('model')[['model', 'HitRate@1: ', 'HitRate@5: ', 'HitRate@10: ', 'HitRate@20: ']])

      model  HitRate@1:   HitRate@5:   HitRate@10:   HitRate@20: 
7   gru4rec     0.093575     0.308659      0.488827      0.603352
6  hgru4rec     0.185754     0.272346      0.307263      0.357542
5        sr     0.234637     0.671788      0.804469      0.871508
3      sr_B     0.256983     0.705307      0.828212      0.871508
0      stan     0.090782     0.668994      0.796089      0.867318
1     vsknn     0.131285     0.553073      0.737430      0.840782
2  vsknn_EB     0.039106     0.600559      0.758380      0.828212
4     vstan     0.131285     0.550279      0.692737      0.734637
      model  HitRate@1:   HitRate@5:   HitRate@10:   HitRate@20: 
7   gru4rec     0.219210     0.413633      0.522076      0.608056
2  hgru4rec     0.251743     0.354764      0.382649      0.438420
4        sr     0.274206     0.666150      0.791634      0.870643
3      sr_B     0.291247     0.688613      0.815647      0.870643
5      stan     0.110767     0.621224      0.754454      0.835786
0     vskn

In [23]:
df.sort_values('Metrics')[['Metrics', 'HitRate@1: ', 'HitRate@20: ']]

,Metrics,HitRate@1:,HitRate@20:
7,gru4rec-momentum=0.1-dropout_p_hidden=0.1-fina...,0.093575,0.603352
6,hgru4rec-final_act=linear-batch_size=50-moment...,0.185754,0.357542
5,sr-weighting=quadratic-steps=20,0.234637,0.871508
3,sr_B-boost_own_sessions=3.1-weighting=quadrati...,0.256983,0.871508
0,stan-sample_size=2500-k=1500-lambda_spw=0.905-...,0.090782,0.867318
1,vsknn-sample_size=5000-weighting=quadratic-k=5...,0.131285,0.840782
2,vsknn_EB-sample_size=1000-weighting=log-weight...,0.039106,0.828212
4,vstan-similarity=vec-k=200-lambda_snh=40-sampl...,0.131285,0.734637


In [2]:
USER_KEY = 'userID'
TIME_KEY = 'timestamp'
ITEM_KEY = 'appID'
SESSION_KEY = 'sessionID'

In [5]:
path_results = '../results/app-level/multiple/drop_on-drop_off/gru4rec_embedding/window_1/'
# path_results = '../results/drop_on/untuned/'
# path_results = '../results/drop_on-drop_off/untuned/'

test_data = pd.read_hdf('../../data/app-level/multiple/' + 'events-1-drop_on-drop_off.hdf', 'test') # make sure this aligns with path_results

In [8]:
results_files = [f for f in os.listdir(path_results) if 'Saver' not in f]
results = pd.DataFrame()
for file in results_files:
    df = pd.read_csv(path_results + file, sep = ';')
    # TBD: extract window number from file name, create new column
    df.drop(['Metrics', 'Saver@50: '], axis=1, inplace=True)
    df.drop(df.filter(regex='Unnamed'), axis=1, inplace=True) # drop 'Unnamed: 24' column containing only NaNs
    df.rename(columns = lambda x : str(x)[:-2], inplace=True) # remove colon and whitespace from all column names
    df.insert(0, 'model', "_".join(file.split('_')[2:-1]))
    results = results.append(df)

In [9]:
results

,model,MAP@1,MAP@5,MAP@10,MAP@20,Precision@1,Precision@5,Precision@10,Precision@20,Recall@1,...,HitRate@1,HitRate@5,HitRate@10,HitRate@20,MRR@1,MRR@5,MRR@10,MRR@20,Coverage@20,Popularity@20
0,gru4rec_128,0.0,0.054402,0.036659,0.023861,0.286325,0.163533,0.103704,0.065812,0.087556,...,0.192308,0.490028,0.579772,0.683761,0.192308,0.301401,0.313027,0.320133,0.853503,0.033256
0,gru4rec_16,0.0,0.052329,0.038012,0.024255,0.314815,0.164672,0.106410,0.066952,0.106706,...,0.217949,0.487179,0.586895,0.683761,0.217949,0.311301,0.323815,0.330560,0.851911,0.036083
0,gru4rec_32,0.0,0.053856,0.036623,0.024361,0.293447,0.160969,0.102849,0.066738,0.104696,...,0.206553,0.472934,0.566952,0.679487,0.206553,0.301330,0.313606,0.321388,0.837580,0.035584
0,gru4rec_64,0.0,0.052729,0.036304,0.024363,0.297721,0.170370,0.105983,0.067236,0.085880,...,0.190883,0.507123,0.572650,0.686610,0.190883,0.304915,0.313298,0.321251,0.874204,0.035555
0,gru4rec,0.0,0.054402,0.036659,0.023861,0.286325,0.163533,0.103704,0.065812,0.087556,...,0.192308,0.490028,0.579772,0.683761,0.192308,0.301401,0.313027,0.320133,0.853503,0.033256
